In [1]:
from occupancy.pipelines.autoencoderkl_3d import AutoEncoderKL3d, AutoEncoderKL3dConfig
import torch
import math
import logging
from torch import nn, Tensor, onnx
vae3d = AutoEncoderKL3d(1, 1, 16, exportable=True)
vae3d.load_state_dict(torch.load('/home/zc2309/workspace/occupancy/models/autoencoderkl-cls1.pt'))

<All keys matched successfully>

In [1]:
256 * 256 * 32 / 1000000

2.097152

In [ ]:
def occ_shuffle(occ: t)

In [3]:
from typing import Optional


tensor_x = torch.rand(1, 1, 256, 256, 32).float()
tensor_y = torch.rand(1, 16, 32, 32, 4).float()
vae3d = vae3d.float()
encoder_onnx = onnx.dynamo_export(vae3d.encoder, tensor_x)
decoder_onnx = onnx.dynamo_export(vae3d.decoder, tensor_y)

In [ ]:
encoder_onnx.save('autoencoderkl-cls1-encoder.onnx')
decoder_onnx.save('autoencoderkl-cls1-decoder.onnx')

In [ ]:
32 * 32 * 32

In [4]:
import torch
import sys
import os
sys.path.append(os.path.dirname(__file__))
torch.hub.load('serizba/salad', 'dinov2_salad')

NameError: name '__file__' is not defined

In [6]:
from torch import nn
import torch
x = torch.randn(1, 3, 32, 32)
conv= nn.Conv2d(3, 6,1)
y = conv(x)
y.shape

torch.Size([1, 6, 32, 32])

In [7]:
from torch import nn
import torch
import torch.nn.functional as F


class VisionTransformer(nn.Module):

    def __init__(self, in_channels: int, out_channels: int, hidden_size: int, patch_size: int, num_layers: int):
        super().__init__()
        self.positional_embeds = nn.Embedding(100000, hidden_size)
        self.register_buffer("positional_ids", torch.arange(100000).unsqueeze(0))
        self.patch_conv = nn.Conv2d(in_channels, hidden_size, patch_size, stride=patch_size)
        self.patch_deconv = nn.ConvTranspose2d(hidden_size, out_channels, patch_size, stride=patch_size)
        self.model = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(
                hidden_size,
                hidden_size // 64,
                hidden_size * 8 // 3,
                activation=F.silu,
                batch_first=True,
                norm_first=True,
            ),
            num_layers,
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        hidden = self.patch_conv(x)
        n_seq = hidden.shape[2] * hidden.shape[3]
        pos_embeds = self.positional_embeds(self.positional_ids[:, :n_seq]).expand(hidden.shape[0], -1, -1)
        output = self.model(hidden.flatten(2).transpose(1, 2) + pos_embeds).transpose(-1, -2).view_as(hidden)
        output = self.patch_deconv(output)
        return output


vit = VisionTransformer(3, 3, 512, 4, 12)
x = torch.randn(1, 3, 256, 256)
y = vit(x)
y.shape

/home/zc2309/mambaforge/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [ ]:
from torch import Tensor, nn
from torchvision import models  # type: ignore
from torchvision.models.feature_extraction import (  # type: ignore
    create_feature_extractor,
)


class VisionTransformer(nn.Module):
    def __init__(
        self,
        feat_dim: int,
        hidden_size: int,
        patch_size: int,
        num_layers: int,
        **kwargs
    ):
        super().__init__()
        self.feat_dim = feat_dim
        self.hidden_size = hidden_size
        self.patch_size = patch_size
        self.num_layers = num_layers
        self.encoder = models.vit_l_16(
            weights=models.ViT_L_16_Weights.IMAGENET1K_SWAG_LINEAR_V1
        )
        self.encoder.requires_grad_(False)
        for i in range(8):
            self.encoder.encoder.layers[-i].requires_grad_(True)
        self.encoder = create_feature_extractor(
            self.encoder, {"encoder.layers.encoder_layer_23.add_1": "encoder_output"}
        )
        self.pre_avg_proj = nn.Linear(hidden_size, feat_dim)
        self.out_norm = nn.InstanceNorm1d(feat_dim)

    def forward(self, x: Tensor) -> Tensor:
        x = self.encoder(x)["encoder_output"]
        x = self.pre_avg_proj(x)
        x = x.mean(dim=-2)
        x = self.out_norm(x)
        return x

In [1]:
# Load model directly
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("openbmb/MiniCPM-2B-dpo-bf16", trust_remote_code=True)

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

configuration_minicpm.py:   0%|          | 0.00/9.77k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-2B-dpo-bf16:
- configuration_minicpm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_minicpm.py:   0%|          | 0.00/67.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/openbmb/MiniCPM-2B-dpo-bf16:
- modeling_minicpm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/5.45G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
torch.manual_seed(0)

path = 'openbmb/MiniCPM-2B-dpo-bf16'
tokenizer = AutoTokenizer.from_pretrained(path)
model = AutoModelForCausalLM.from_pretrained(path, torch_dtype=torch.bfloat16, device_map='cuda', trust_remote_code=True)

responds, history = model.chat(tokenizer, "山东省最高的山是哪座山, 它比黄山高还是矮？差距多少？如果你需要查资料验证，随时可以告诉我。", temperature=0.8, top_p=0.8)
print(responds)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


山东省最高的山是泰山，海拔1545米。而黄山位于中国安徽省，海拔1864米。因此，泰山比黄山矮，相差319米。


In [8]:
y.shape

torch.Size([1, 3, 256, 256])

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype=torch.bfloat16, device_map='auto')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
input = """
Instruct: Please translate this from English to Chinese: "Hello, how are you?"
Output: 
"""
input_ids = tokenizer(input, return_tensors="pt").input_ids.to('cuda')
output = model.generate(input_ids, do_sample=True, max_length=50, num_return_sequences=3, temperature=0.7)
print(tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Instruct: Please translate this from English to Chinese: "Hello, how are you?"
Output: 
你好，你怎么样？

